In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
import numpy as np
import evaluate
from collections import defaultdict
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction, AutoTokenizer, TrainingArguments, Trainer, \
    DataCollatorForTokenClassification, DataCollatorWithPadding, \
    AutoModelForTokenClassification, AutoModelForSequenceClassification

from data_loader import DataLoader

# Выгрузка набора данных

In [26]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer_params = dict(truncation=True)

loader = DataLoader("./data", tokenizer, tokenizer_params)
datasets = loader.get_data_for_experiment(domain="all", regime="large")
info_columns = ["text", "intents", "tagger", "tokens"]

In [11]:
args = TrainingArguments("distilbert-finetuned",
                        learning_rate=1e-5,
                        optim="adamw_torch_fused", # немного оптимизированный под ГПУ оптимизер
                        per_device_train_batch_size=16,
                        per_device_eval_batch_size=128, # поскольку на шаге eval не нужно считать loss, граф вычислений не строится и обычно можно использовать батч побольше
                        num_train_epochs=7,
                        weight_decay=0.01, # https://medium.com/unpackai/stay-away-from-overfitting-l2-norm-regularization-weight-decay-and-l1-norm-regularization-795bbc5cf958
                        warmup_steps = 100, # число шагов (батчей) за которые lr увеличивается от 0 до заданного нами
                        lr_scheduler_type = 'linear', # эта штука "постепенно" скручивает learning rate по ходу эпох. linear - дефолтное значение
                        evaluation_strategy="steps", # выбираем стратегию оценки качества модели - каждые n батчей (как альтернатива - по эпохам)
                        eval_steps=200, # раз в сколько батчей оцениваем
                        seed=42, # просто немного для стабильности
                        fp16=True, #  очень вполезная штука, заметно скоряет обучение, хотя работает только на GPU
                        logging_strategy='steps',
                        logging_steps=200, # как часто считаем лос на трейне)
                        save_strategy="no"
)

# Задача тегирования слотов

In [12]:
metric = evaluate.load('seqeval')
label_names = list(loader.tag2index.keys())

def compute_metrics(eval_preds):
    logits, labels = eval_preds

    predictions = np.argmax(logits, axis=-1)

    true_labels = [[label_names[l] for l in label if l!=-100] for label in labels]

    true_predictions = [[label_names[p] for p,l in zip(prediction, label) if l!=-100] 
                        for prediction, label in zip(predictions, labels)]

    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

    return {"precision": all_metrics['overall_precision'],
            "recall": all_metrics['overall_recall'],
            "f1": all_metrics['overall_f1'],
            "accuracy": all_metrics['overall_accuracy']}

In [33]:
tagger_result = defaultdict(list)

for fold_dataset in datasets.values():
    model = AutoModelForTokenClassification.from_pretrained(
        "distilbert-base-uncased",
        id2label=loader.index2tag,
        label2id=loader.tag2index
    ).to('cuda')
    
    train_dataset = fold_dataset['train'].remove_columns(info_columns + ["classification_labels"]).rename_column("tagging_labels", "labels")
    test_dataset = fold_dataset['test'].remove_columns(info_columns + ["classification_labels"]).rename_column("tagging_labels", "labels")

    trainer = Trainer(model=model,
                    args=args,
                    train_dataset = train_dataset,
                    eval_dataset = test_dataset,
                    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer),
                    compute_metrics=compute_metrics,
                    tokenizer=tokenizer)

    trainer.train()
    for name, val in trainer.evaluate().items():
        if val != 'epoch':
            tagger_result[name].append(val)

for name, val in tagger_result.items():
    print(name, np.mean(val))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
200,1.434000,0.542689,0.272727,0.222222,0.244898,0.876749
400,0.386500,0.347166,0.410256,0.423280,0.416667,0.910060
600,0.251500,0.281186,0.468599,0.513228,0.489899,0.925383
800,0.199500,0.256150,0.518519,0.592593,0.553086,0.931712
1000,0.165400,0.243599,0.532110,0.613757,0.570025,0.938041
1200,0.158100,0.240034,0.545024,0.608466,0.575000,0.941372


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
200,1.530900,0.416180,0.340278,0.267760,0.299694,0.905299
400,0.376500,0.250426,0.463542,0.486339,0.474667,0.935717
600,0.260600,0.207767,0.540000,0.590164,0.563969,0.945437
800,0.198300,0.182872,0.588542,0.617486,0.602667,0.951395
1000,0.169300,0.172753,0.627027,0.633880,0.630435,0.953277
1200,0.152300,0.170666,0.607330,0.633880,0.620321,0.954531


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
200,1.517200,0.532347,0.293750,0.224880,0.254743,0.876196
400,0.374700,0.331449,0.442922,0.464115,0.453271,0.916023
600,0.252400,0.268936,0.543103,0.602871,0.571429,0.934856
800,0.193600,0.233684,0.591304,0.650718,0.619590,0.941957
1000,0.167600,0.221599,0.616071,0.660287,0.637413,0.946897
1200,0.143300,0.216085,0.606987,0.665072,0.634703,0.948441


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
200,1.524500,0.466254,0.261538,0.200000,0.226667,0.885667
400,0.374900,0.292795,0.359551,0.376471,0.367816,0.926000
600,0.246700,0.241171,0.505556,0.535294,0.520000,0.939000
800,0.193500,0.215619,0.554348,0.600000,0.576271,0.948667
1000,0.162400,0.207138,0.550265,0.611765,0.579387,0.951000
1200,0.150700,0.202342,0.544503,0.611765,0.576177,0.954000


eval_loss 0.19529844373464583
eval_precision 0.6173427144167403
eval_recall 0.6616213122720609
eval_f1 0.6384978663631674
eval_accuracy 0.9526286757642805
eval_runtime 0.19548000000000001
eval_samples_per_second 1578.8169
eval_steps_per_second 15.383700000000001
epoch 7.0


# Задача классификации интентов

In [45]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.15):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [47]:
args = TrainingArguments("distilbert-finetuned",
                        learning_rate=1e-5,
                        optim="adamw_torch_fused", # немного оптимизированный под ГПУ оптимизер
                        per_device_train_batch_size=16,
                        per_device_eval_batch_size=128, # поскольку на шаге eval не нужно считать loss, граф вычислений не строится и обычно можно использовать батч побольше
                        num_train_epochs=15,
                        weight_decay=0.01, # https://medium.com/unpackai/stay-away-from-overfitting-l2-norm-regularization-weight-decay-and-l1-norm-regularization-795bbc5cf958
                        warmup_steps = 100, # число шагов (батчей) за которые lr увеличивается от 0 до заданного нами
                        lr_scheduler_type = 'linear', # эта штука "постепенно" скручивает learning rate по ходу эпох. linear - дефолтное значение
                        evaluation_strategy="steps", # выбираем стратегию оценки качества модели - каждые n батчей (как альтернатива - по эпохам)
                        eval_steps=200, # раз в сколько батчей оцениваем
                        seed=42, # просто немного для стабильности
                        fp16=True, #  очень вполезная штука, заметно скоряет обучение, хотя работает только на GPU
                        logging_strategy='steps',
                        logging_steps=200, # как часто считаем лос на трейне)
                        save_strategy="no"
)

In [48]:
classifier_result = defaultdict(list)

for fold_dataset in datasets.values():
    model = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", 
        problem_type="multi_label_classification", 
        num_labels=len(loader.index2intent.keys()), 
        id2label=loader.index2intent, 
        label2id=loader.intent2index
    ).to('cuda')
    
    train_dataset = fold_dataset['train'].remove_columns(info_columns + ["tagging_labels"]).rename_column("classification_labels", "labels")
    test_dataset = fold_dataset['test'].remove_columns(info_columns + ["tagging_labels"]).rename_column("classification_labels", "labels")

    trainer = Trainer(model=model,
                    args=args,
                    train_dataset = train_dataset,
                    eval_dataset = test_dataset,
                    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                    compute_metrics=compute_metrics,
                    tokenizer=tokenizer)

    trainer.train()
    for name, val in trainer.evaluate().items():
        if val != 'epoch':
            classifier_result[name].append(val)

for name, val in classifier_result.items():
    print(name, np.mean(val))

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
200,0.480600,0.240150,0.079361,0.570263,0.000000
400,0.177700,0.143087,0.172879,0.569471,0.006579
600,0.139500,0.134577,0.181656,0.567694,0.013158
800,0.132800,0.127528,0.298201,0.623162,0.108553
1000,0.125700,0.118198,0.405728,0.684469,0.141447
1200,0.116900,0.109975,0.446964,0.701589,0.154605
1400,0.108800,0.103476,0.477237,0.722568,0.157895
1600,0.102100,0.098108,0.506709,0.736135,0.184211
1800,0.098600,0.094011,0.542722,0.753528,0.207237
2000,0.094600,0.091234,0.554949,0.763244,0.217105


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
200,0.479300,0.237757,0.080583,0.573563,0.000000
400,0.178200,0.144390,0.166275,0.566584,0.009772
600,0.139100,0.135805,0.169872,0.567307,0.013029
800,0.134200,0.130098,0.260943,0.607289,0.110749
1000,0.126800,0.121629,0.376972,0.670937,0.146580
1200,0.118600,0.114199,0.428693,0.682931,0.166124
1400,0.110000,0.106921,0.449074,0.710913,0.159609
1600,0.103600,0.101838,0.490980,0.728706,0.179153
1800,0.099300,0.098082,0.510938,0.739537,0.192182
2000,0.093400,0.095168,0.532081,0.749772,0.205212


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
200,0.480200,0.237426,0.077073,0.565179,0.000000
400,0.177800,0.143170,0.160126,0.563935,0.009646
600,0.140600,0.133919,0.159500,0.563803,0.009646
800,0.135200,0.127934,0.246039,0.592831,0.099678
1000,0.126300,0.119437,0.386489,0.677186,0.115756
1200,0.119000,0.110803,0.442636,0.698791,0.128617
1400,0.111500,0.104207,0.495604,0.728089,0.157556
1600,0.104800,0.098666,0.524073,0.744965,0.167203
1800,0.099700,0.094573,0.535385,0.756674,0.167203
2000,0.094500,0.091521,0.553544,0.770927,0.163987


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
200,0.478900,0.235342,0.081107,0.579582,0.000000
400,0.176500,0.142114,0.161989,0.564857,0.009934
600,0.140300,0.133721,0.163035,0.565075,0.009934
800,0.133500,0.127894,0.264865,0.605604,0.129139
1000,0.127100,0.119429,0.400998,0.678754,0.142384
1200,0.117900,0.111185,0.439105,0.697933,0.158940
1400,0.110400,0.104241,0.483165,0.716155,0.182119
1600,0.103100,0.098903,0.513036,0.730615,0.198675
1800,0.098600,0.094453,0.531932,0.748624,0.208609
2000,0.094500,0.091560,0.571429,0.765852,0.228477


eval_loss 0.0880474478006363
eval_f1 0.5691238609274476
eval_roc_auc 0.7722500363992733
eval_accuracy 0.21476230624151418
eval_runtime 0.15300999999999998
eval_samples_per_second 2022.3902999999998
eval_steps_per_second 19.7069
epoch 15.0
